###  Modelo que implementa el GoL 

Sin hacer caso a mis tutores, voy a seguir el capitulo 12 del libro "Hands on Machine Learning with Scikit-Learn Keras y TensorFlow",
y con ayuda de ChatGPT, vamos a intentar crear una "Custom Layer" que implemente el GoL. He leido el cap y creo haberme enterado, 
lo cual me ha motivado a hacer esto, pues sería el aproach correcto del TFG. Independientemente de esto, el modelo de referencia
sigue siendo el mismo, puesto que solo voy a incorporar la capa.

#### Muestra de que entiendo qué está pasando:

Nosotros queremos crear una capa que no está definida por defecto en TensorFlow o en Keras, puesto que queremos incorporar una capa que juegue al GoL.
En consecuencia, tenemos que estudiar cómo crear esta layer y, lo que és más importante, estudiar cómo implementa en el modelo. Para la implementación se usan APIs
de TensorFlow y Keras, por lo que no hay que rallarse, tan solo hay que usar los comandos asociados. Para la creación eso si que tiene más fiesta, pero lo importante:

- La capa simplemente implementa el GoL, no realiza predicciones ni nada => No hace falta definir un "build" en esta clase.
- Vamos a guardar los pesos del modelo, entonces hay que definir un get_config.

Lo más importante, es que no vamos a jugar al GoL clásico puesto que NO es diferenciable. Esta característica es vital para que el modelo pueda aprender => Debemos buscar
la forma de hacer que el Juego de la Vida (discreto) sea continuo. Como primera propuesta, y atendiendo a que el GoL clasifica los estados en dos clases (1 o 0), vamos a usar
la función sigmoide. ¿Cómo se hace esto? Implementamos la función sigmoide centrada con respecto al valor de la vecindad.

1. Condición de supervivencia: si el estado de la célula es x $\approx$ 1, entonces implementamos una función sigmoide centrada en el intervalo [2,3], de tal forma que es no nula en este intervalo y
nula fuera de este.

$$

 \sigma(x,n) = \frac{x}{1 + e^{+50(n - 3)(n- 2)}} 

$$
Esta definición de la sigmoide la transforma en una gaussiana => Podemos hacer uso de la sigmoide de tensorflow y convertirla en una gaussiana para jugar al GoL. EL Factor 50 es para 
que su forma se más cuadrada:

![](Sigmoides/SurvivalSigmoid.png)


2. Condición de nacimiento: si el estado de la célula es x $\approx$ 0, entonces implementamos una función sigmoide tal que tenga un maximo en 3, decayendo conforme se aleja de este valor.
$$
\sigma(x,n) = \frac{2(1 - x)}{1 + e^{+50(n - 3)^2}}

$$

El factor 2 es para asegurar que la sigmoide alcance el valor 1 y el 50 para adelgazar la función:

![](Sigmoides/BornSigmoid.png)


La ventaja de todo este procedimiento es que solo necesitados tableros finales en los datasets



In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras as ks
import random
import tensorflow as tf
from keras import Input, Model
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.losses import BinaryFocalCrossentropy

# 1. Define la semilla
SEED = 42  
 
# 2. Python built-in random
random.seed(SEED)

# 3. NumPy
np.random.seed(SEED)

# 4. TensorFlow
tf.random.set_seed(SEED)

# (Opcional) Para TensorFlow más determinismo en operaciones GPU:
os.environ['TF_DETERMINISTIC_OPS'] = '1'

path = "../Datos"
datos = []
for dirnames,_,filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('.xlsx'):
            datos.append(os.path.join(dirnames,filename))

print(datos)

['../Datos\\test.xlsx', '../Datos\\train.xlsx']


**1. Funciones de las métricas:**

In [12]:
# ---------------- Para el TFG ----------------
def Accuracy(y_true, y_pred):
    y_pred_rounded = K.round(K.clip(y_pred, 0, 1))
    correct = K.equal(y_true, y_pred_rounded)
    return K.mean(K.cast(correct, K.floatx()))

def Precision(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    tp = K.sum(y_true * y_pred_pos)
    predicted_positives = K.sum(y_pred_pos)
    return tp / (predicted_positives + K.epsilon())

def Specificity(y_true, y_pred):
    y_pred_neg = 1 - K.round(K.clip(y_pred, 0, 1))
    y_true_neg = 1 - y_true
    tn = K.sum(y_true_neg * y_pred_neg)
    possible_negatives = K.sum(y_true_neg)
    return tn / (possible_negatives + K.epsilon())

# ---------------- Extras ----------------
def Recall(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    tp = K.sum(y_true * y_pred_pos)
    possible_positives = K.sum(y_true)
    return tp / (possible_positives + K.epsilon())

def F1_score(y_true, y_pred):
    prec = Precision(y_true, y_pred)
    rec = Recall(y_true, y_pred)
    return 2 * (prec * rec) / (prec + rec + K.epsilon())

def Hamming_loss(y_true, y_pred):
    mismatches = K.not_equal(K.round(K.clip(y_pred, 0, 1)), y_true)
    return K.mean(K.cast(mismatches, K.floatx()))


**2. Creamos la capa del GoL y el modelo:**

In [13]:
# Capa diferenciable del Game of Life
class DifferentiableGoL(tf.keras.layers.Layer):
    def __init__(self, steps=1, name=None):
        super().__init__(name=name)
        self.steps = steps
        kernel = [[1, 1, 1],
                  [1, 0, 1],
                  [1, 1, 1]]
        self.kernel = tf.constant(kernel, dtype=tf.float32)
        self.kernel = tf.reshape(self.kernel, [3, 3, 1, 1])  # para tf.nn.conv2d

    def call(self, x):
        for _ in range(self.steps):
            neighbors = tf.nn.conv2d(x, self.kernel, strides=1, padding='SAME')
            survive = x * tf.sigmoid(-50*((neighbors - 2) * (neighbors - 3))) # Condición de supervivencia
            born = (1 - x)* 2 * tf.sigmoid(-50*(neighbors - 3) ** 2) # Condición de nacimiento
            x = tf.clip_by_value(survive + born, 0, 1)
        return x
    
    def get_config(self):
        config = super().get_config()
        config.update({"steps": self.steps})
        return config

# Constructor del modelo
def create_gol_model(n_hidden_convs=2, n_hidden_filters=128, kernel_size=5, steps=1):
    """
    Crea un modelo CNN para Reverse Game of Life con capa diferenciable GoL.

    Inputs:
        n_hidden_convs: Número de capas convolucionales intermedias
        n_hidden_filters: Número de filtros por capa
        kernel_size: Tamaño del kernel convolucional
        steps: Número de pasos del GoL (equivalente a delta)

    Outputs:
        Un modelo tf.keras.Model compilado listo para entrenar
    """
   
    input_final = Input(shape=(20, 20, 1), name='input_final')

    x = Conv2D(n_hidden_filters, kernel_size, padding='same', activation='relu')(input_final)
    x = BatchNormalization()(x)

    for _ in range(n_hidden_convs):
        x = Conv2D(n_hidden_filters, kernel_size, padding='same', activation='relu')(x)
        x = BatchNormalization()(x)

    predicted_init = Conv2D(1, kernel_size, padding='same', activation='sigmoid',
                            name='predicted_init')(x)

    predicted_final = DifferentiableGoL(steps=steps, name='predicted_final')(predicted_init)

    model = Model(inputs=input_final, outputs=[predicted_final, predicted_init], name='GoL_Model')
    
    focal_loss = BinaryFocalCrossentropy(
        alpha=0.75,      # valor mayor a 0.5 da mayor peso a la clase positiva (1)
        gamma=2,
        from_logits=False)
    
    model.compile(
        optimizer='adam',
        loss={'predicted_final': focal_loss},
        metrics= [Accuracy, Precision, Specificity, Recall, F1_score, Hamming_loss])

    return model

**3. Cargamos los datos:**

In [14]:
# Cargo los datos:
train = pd.read_excel(datos[1], sheet_name = 'train', header = 0)

# Creo una lista con los nombres de los headers de los tableros:
stops = [f'stop.{i}' for i in range(1,401)]

**4. Entrenamiento del Modelo:**

In [15]:
# Entrenamiento del modelo delta = 1. Se aplica un bucle para permitir entrenar otros modelos.

models = [] # Almacén de modelos (parámetros optimizados)
historial = [] # Almacén de historiales (loss, accurracy)
i=1
for i in range(1,2):
    # Obtenemos los datos para cada delta:
    delta_i = train[train['delta'] == i]
    finales_i = np.reshape(delta_i[stops].values, (-1,20,20,1), order='F')
    
    # Entrenamos al modelo
    model = create_gol_model(n_hidden_convs=6, n_hidden_filters=256, kernel_size=5, steps=i)
    es = EarlyStopping(monitor='loss', patience=9, min_delta=0.001)
    history = model.fit(finales_i, {'predicted_final': finales_i}, epochs=50, verbose=1)
    models.append(model)
    historial.append(history)

Epoch 1/50
309/309 [==============================] - 49s 154ms/step - loss: 1.5069 - predicted_final_loss: 1.5069 - predicted_final_Accuracy: 0.8199 - predicted_final_Precision: 0.1831 - predicted_final_Specificity: 0.9336 - predicted_final_Recall: 0.1040 - predicted_final_F1_score: 0.1175 - predicted_final_Hamming_loss: 0.1801 - predicted_init_Accuracy: 0.0000e+00 - predicted_init_Precision: 0.0000e+00 - predicted_init_Specificity: 0.0000e+00 - predicted_init_Recall: 0.0000e+00 - predicted_init_F1_score: 0.0000e+00 - predicted_init_Hamming_loss: 0.0000e+00
Epoch 2/50
309/309 [==============================] - 48s 156ms/step - loss: 0.4968 - predicted_final_loss: 0.4968 - predicted_final_Accuracy: 0.8201 - predicted_final_Precision: 0.2443 - predicted_final_Specificity: 0.9243 - predicted_final_Recall: 0.1602 - predicted_final_F1_score: 0.1662 - predicted_final_Hamming_loss: 0.1799 - predicted_init_Accuracy: 0.0000e+00 - predicted_init_Precision: 0.0000e+00 - predicted_init_Specificit

**5. Guardamos todo para reutilizarlo cuando queramos:**  De esta forma no tenemos que ejecutar todo el notebook cada vez.

In [17]:
# Convertimos el diccionario de historial en un DataFrame:
df_history1 = pd.DataFrame(historial[0].history)

# Guardamos el DataFrame en un archivo Excel:
df_history1.to_excel('../Modelo V2 Reverse FBCE/Historial Entrenamiento/historialFBCE_delta_1.xlsx', index=False)

# Guardamos los modelos
for i, model in enumerate(models):
    model.save(f"../Modelo V2 Reverse FBCE/Pesos Modelos/modeloFBCE_delta_{i+1}", save_format='tf')

INFO:tensorflow:Assets written to: ../Modelo V2 Reverse FBCE/Pesos Modelos/modeloFBCE_delta_1\assets


INFO:tensorflow:Assets written to: ../Modelo V2 Reverse FBCE/Pesos Modelos/modeloFBCE_delta_1\assets
